In [1]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import *
from pyspark.sql.functions import col
from pyspark.sql.window import Window
from pyspark.ml.feature import HashingTF, Normalizer
from pyspark.sql.types import StringType, ArrayType, DoubleType, IntegerType

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
120,application_1575505342345_0165,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


# Validación de Data previa ( INPUT )

Antes de procesar la información, realizamos la validación de la información previa. La validación pasa por el orden de la interfaz que ha tenido mas problemas. Esto para asegurar que no se va tener problemas en el Train / Predict. 

### Rutas de Parquets files ha validar

In [2]:
####Paramters
str_matrizcampana = 's3://belc-bigdata-functional-dlk-qas/analitico/GlueDatabase/functional-offline/dwh_dmatrizcampana/'
str_fvtaproebecam = 's3://belc-bigdata-functional-dlk-qas/analitico/GlueDatabase/functional-offline/dwh_fvtaproebecam/'
str_tipooferta = 's3://belc-bigdata-functional-dlk-qas/analitico/GlueDatabase/functional-offline/dwh_dtipooferta/'
str_offert = 's3://belc-bigdata-landing-dlk-qas/lan-ganamas/input/cons/'
str_tiempocampana = 's3://belc-bigdata-functional-dlk-qas/analitico/GlueDatabase/functional/dwh_dtiempocampana/'

### Trae Información

In [3]:
df_tiempocampana = spark.read.parquet(str_tiempocampana)
df_dmatrizcampana = spark.read.parquet(str_matrizcampana)

In [4]:
ANIOCAMPAN_INI = '201910'
ANIOCAMPAN_FIN = '202004'

In [5]:
df_campana = df_tiempocampana.filter( 
    (col('codpais') != 'US') & 
    (col('aniocampana') >= ANIOCAMPAN_INI) & (col('aniocampana') <= ANIOCAMPAN_FIN)
).select('codpais', 'aniocampana')

In [6]:
def CalculaAnioCampana(codpais, aniocampana, delta):
    if codpais != 'PR':
        anio, camp = int(int(aniocampana)/100), int(int(aniocampana)%100) + delta
        if camp > 18: camp = camp-18; anio = anio+1;          
        if camp < 1: camp = 18 + camp; anio = anio-1;
        if camp < 1 or camp > 18: return CalculaAnioCampana(codpais, str(anio) + '00', camp)
        return (str(anio) + str('00'+str(camp))[-2:])
    else:
        return aniocampana

## DMatrizCampana

Valida información:
* Se buscan ofertas digitales ( codtipooferta>'200'), que esten aspeados (codventa != '00000')
* PartitionBy (max_total) para buscar el total tipos de ofertas por campaña

Los resultados que se obtengan son Pais/Campaña que no tienen mal cargadas los tipos de ofertas digitales. 

In [ ]:
#Condiciones
df_matriz1 = df_dmatrizcampana.filter( 
            (col('codtipooferta') >= '200') & (col('codventa') != '00000') & 
            (col('codpais') != 'US') & 
            (col('aniocampana') >= ANIOCAMPAN_INI) & (col('aniocampana') <= ANIOCAMPAN_FIN) 
        ).groupBy('codpais','aniocampana','codtipooferta')\
        .count().withColumnRenamed('count','total')\
        .sort(['codpais', 'aniocampana'])

#MaxTotal PartitionBy
window = Window.partitionBy('codpais','aniocampana' )

df_matriz = df_campana.join(df_matriz1,['codpais', 'aniocampana'],'left')\
        .select('codpais', 'aniocampana', 'codtipooferta','total').na.fill({'total': 0})\
        .withColumn("max_total",count('total').over(window))

df_matriz.filter( 
    (col('max_total') <= 3)
).orderBy(['aniocampana', 'codpais']).show(100)

Validación unitaria.

In [ ]:
df_dmatrizcampana.filter( 
    (col('codpais') == 'PR') & (col('aniocampana') == '201912') 
    ).select('codpais', 'aniocampana', 'codtipooferta', 'codventa').show()


## FvtaProEbeCam

Valida información:
* Las campañas cargadas.


In [ ]:
df_fvtaproebecam = spark.read.parquet(str_fvtaproebecam)

In [ ]:
df_fvtaproebecam.filter( 
        (col('codpais') != 'US') & 
        (col('aniocampana') >= ANIOCAMPAN_INI) & (col('aniocampana') <= ANIOCAMPAN_FIN) 
        ).groupBy('codpais','aniocampana').\
        agg(sum(col('realvtamnneto')).alias('realvtamnneto')).\
        withColumn('aniocampana_1', CalculaAnioCampana(col('codpais'), col('aniocampana'), 1)).\
        select('codpais','aniocampana', 'aniocampana_1', 'realvtamnneto')
sort(['codpais', 'aniocampana']).show()
            

In [ ]:
 CalculaAnioCampana(df_fvtaproebecam['codpais'], df_fvtaproebecam['aniocampana'], df_fvtaproebecam['aniocampana'] )

## DTipoOferta

In [ ]:
def CalculaAnioCampana(codpais, aniocampana, delta):
    if codpais != 'PR':
        anio, camp = int(int(aniocampana)/100), int(int(aniocampana)%100) + delta
        if camp > 18: camp = camp-18; anio = anio+1;          
        if camp < 1: camp = 18 + camp; anio = anio-1;
        if camp < 1 or camp > 18: return CalculaAnioCampana(codpais, str(anio) + '00', camp)
        return (str(anio) + str('00'+str(camp))[-2:])
    else:
        return aniocampana

In [ ]:

print (CalculaAnioCampana('CO', '201911', -44) )

In [ ]:

anio, camp = int(int('201911')/100), int(int('201911')%100) + 1

print(anio, camp)

In [ ]:
int(int('201911')%100)

In [ ]:
df_tipooferta = spark.read.parquet(str_tipooferta)

In [ ]:
df_tipooferta.filter((col('codpais') == 'CO') & (col('codtipooferta') >= '200')).select('codtipooferta', 'destipooferta').show(40)

In [ ]:
df_offerta = spark.read.parquet(str_offert)

In [ ]:
df_offerta.show(5)

In [ ]:
df_offerta.groupBy('country', 'campaign', 'palanca').count().sort(['country', 'campaign', 'palanca']).show(50)

# Validación Resultados ( OUTPUT )

In [ ]:
df_output = spark.read.parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/output/year=2019/month=10/day=29/country=PE/")

In [ ]:
df_output = spark.read.parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/output/year=2019/month=12/day=02/")

In [ ]:
df_output.show(5)

In [ ]:
df_output.groupBy('country','campaign').count().sort(['country', 'campaign']).show()

In [ ]:
df_new = df_output.groupBy('country', 'campaign','codebelista').agg(max(col('score')).alias('score_top'))

df_new.groupBy('country','campaign').agg(min(col('score_top')).alias('min_score'), max(col('score_top')).alias('max_score')).show()



In [ ]:

#MaxTotal PartitionBy
window = Window.partitionBy('country','campaign')

df_new = df_output.groupBy('country','campaign','codebelista')\
    .agg(max(col('score')).alias('score_top'))\
    .groupBy('country','campaign', 'score_top').count().withColumnRenamed('count','total')\
    .withColumn('suma',sum('total').over(window))\
    .withColumn('Perc', round( ((col('total') / col('suma') ) ), 4) )\
    .sort(['country','campaign', 'score_top'])

In [ ]:
df_new.show(100)

In [ ]:
import pip
pip.main(['install', 'pandas'])

In [ ]:
df_new_pd = df_new.toPandas()

df_new_pd.withColumn('target', concat(col('country'), lit('_'), col('campaign')) )\
    .select('target','Perc')\
    .plot(kind='bar')



In [ ]:
df_new[['score_top', 'count']] = df_new[['score_top', 'count']].astype(int)

In [ ]:
df_new['period'] = df_new[['country', 'campaign']].apply(lambda x: '_'.join(x), axis=1)

In [ ]:
df_new["period"] = df_new["country"].map(str) + df_new["campaign"].map(str)

## Reading dproducto table

In [ ]:
s3_loc = 's3://belc-bigdata-functional-dlk-qas/analitico/GlueDatabase/functional-offline/dwh_dproducto/'
df_dproducto = spark.read.parquet(s3_loc)\
                              .drop('cuc','codcategoria','codsubcategoria','codtipo','codsubtipo') \
                              .filter(col('codcuc') != '').cache() \
                              .withColumnRenamed('codcategoria_homo','codcategoria')\
                              .withColumnRenamed('codsubcategoria_homo', 'codsubcategoria')\
                              .withColumnRenamed('codtipo_homo', 'codtipo')\
                              .withColumnRenamed('codsubtipo_homo', 'codsubtipo')

In [ ]:
df_dproducto.write.mode('overwrite')\
    .parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/inter_input/df_dproducto/")